In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pylab

# This function reads the gro file, looks for all the strings with that contain the name of the slab residue
# and save the data to the output in the format: ['Atom name and atom number', 'atom number', 'X', 'Y', 'Z']
# Note that gro file should not contain velocities!

def ReadGRO(filename, resname):
    
    lines = []
    slab_gmx = [] 
    file = open(filename)
    
    row_normal_width = 6 # resname atomname atomnumber X Y Z
    row_reduced_width = 5 # resname atomname-atomnumber X Y Z (happens when atomname and atomnumber are too long)
    
    for line in file:
        lines.append(line.split())
    file.close()
    
    # go through the rows in lines list (correspond to rows in the input file)
    # check the number of elements in row
    # save only X Y Z coordinates
    for row in range(len(lines)):
        if resname in lines[row][0]:
            if len(lines[row]) == row_normal_width:
                slab_gmx.append(lines[row][3:6])
            elif len(lines[row]) == row_reduced_width:
                slab_gmx.append(lines[row][2:5])
            else:
                print(lines[row])
                print("Number of row elements error")
    
    # convert all strings to floats
    for atom in slab_gmx:
        for coordinate in range(len(atom)):
            atom[coordinate] = float(atom[coordinate])
           
    return(np.array(slab_gmx))
    

In [2]:
def SlabGeometry(filename, resname):
    
    # Get slab coordinates and convert them into a numpy array
    slab = np.array(ReadGRO(filename, resname))
    
    # Get the extreme slab coordinates and save them to the corresponding X, Y, Z lists:
    
    X = []
    X.append(np.amin(slab.T[0]))
    X.append(np.amax(slab.T[0]))
    
    Y = []
    Y.append(np.amin(slab.T[1]))
    Y.append(np.amax(slab.T[1]))
    
    Z = []
    Z.append(np.amin(slab.T[2]))
    Z.append(np.amax(slab.T[2]))
    
    # Note that the coordinates are in nm
    
    return(X, Y, Z)
    

In [3]:
def GenerateCGbeadCenters(filename, resname, x_points, y_points, z_points):
    
    # Get slab extreme coordinates [[x_min, x_max],[y_min, y_max],[z_min, z_max]]
    slab_geometry = SlabGeometry(filename, resname)
    
    # Calculate x, y and z spacings
    x_spacing = (slab_geometry[0][1] - slab_geometry[0][0]) / x_points
    y_spacing = (slab_geometry[1][1] - slab_geometry[1][0]) / y_points
    z_spacing = (slab_geometry[2][1] - slab_geometry[2][0]) / z_points
    
    spacings = np.array([x_spacing, y_spacing, z_spacing])
    
    # Generate grid points
    grid = []
    for x in range(x_points):
        for y in range(y_points):
            for z in range(z_points):
                grid.append(np.array([float(x),float(y),float(z)]))
    
    # Transform it to a numpy array
    grid_np = np.array(grid)
    
    # Convert the grid to the original coordinate system...
    # ...and move each point by half of spacing
    # We want them to become CG bead centers after all!
    grid_np.T[0] *= x_spacing
    grid_np.T[0] += (slab_geometry[0][0] + x_spacing/2)
    
    grid_np.T[1] *= y_spacing
    grid_np.T[1] += (slab_geometry[1][0] + y_spacing/2)
    
    grid_np.T[2] *= z_spacing
    grid_np.T[2] += (slab_geometry[2][0] + z_spacing/2)
    
    
    return(grid_np, spacings)
    
    

In [4]:
def CGbeadHistogram(AllCGbead_AtomIndexes, Nbins):
    atoms_in_CGbead = []
    for CGbead in AllCGbead_AtomIndexes:
        atoms_in_CGbead.append(len(CGbead))
    atoms_in_CGbead = np.array(atoms_in_CGbead)
    fig = plt.figure(figsize=(12,7))
    plt.ylabel('Count', size=20)
    plt.xlabel('Number of atoms per CG bead', size=20)
    pylab.yticks(fontsize=20)
    pylab.xticks(fontsize=20)
    plt.grid(b=True, which='major' , linestyle='--', color= 'grey', linewidth=1, zorder=1)
    plt.grid(b=True, which='minor', linestyle='--', color= 'grey', linewidth=1, zorder=1)
    plt.hist(atoms_in_CGbead, bins = Nbins)
    plt.show()
    
    return(None)

In [5]:
def outputGRO(filename, resname, atomname, x_points, y_points, z_points):
    atoms = np.array(ReadGRO(filename, resname))
    CGbeads_data = GenerateCGbeadCenters(filename, resname, x_points, y_points, z_points)
    CGbeads = CGbeads_data[0]
    CGbeads = np.around(CGbeads, decimals=3)
    
    with open(filename, 'r') as file:
        lines = file.read().splitlines()
        pbc = lines[-1].split()
    
    
    filename = resname + "_" + str(len(CGbeads)) + "CGbeads.gro"
    file_header = "resname: " + resname + "; " + str(len(CGbeads))+ " " + "CG beads" + "\n" + str(len(CGbeads))
    
    gro_file = open(filename, 'w+')
    gro_file.write(file_header + '\n')
    
    for CGbead_index in range(len(CGbeads)):
        gro_file.write(format(resname).rjust(9))
        gro_file.write(format(atomname + str(CGbead_index+1)).rjust(6))
        gro_file.write(format(str(CGbead_index+1)).rjust(5))
        for axis_index in range(len(CGbeads[CGbead_index])):
            gro_file.write(format(CGbeads[CGbead_index][axis_index], '#.3f').rjust(8))
        gro_file.write("\n")
    
    for axis_index in range(len(pbc)):
        gro_file.write(format(float(pbc[axis_index]), '#.5f').rjust(10))
    gro_file.write("\n")
    gro_file.close()
    print(filename, "saved.")
    return
    

In [6]:
# Find CG beads that belong to the surface and first layers and bulk

def FindSlabLayers(CGbeads, AllCGbead_AtomIndexes):
    
    # Generate an array which contains unique Z-coordinate values
    values_z = np.unique(CGbeads.T[2])

    # Surface CG beads have either minimum or maximum values of Z-coordinate 
    surface_z = [values_z[0], values_z[len(values_z)-1]]
    # First subsurface CG beads have next minimum or next maximum values of Z-coordinate
    first_layer_z = [values_z[1], values_z[len(values_z)-2]]

    # Set coordinate tolerance value for comparing floats
    tol = 1e-5

    # Construct an array of surface CG beads indexes
    CGbeads_surface_indexes = []
    for CGbead_index in range(len(CGbeads)):
        if surface_z[0] - tol <= CGbeads[CGbead_index][2] <= surface_z[0] + tol:
            CGbeads_surface_indexes.append(CGbead_index)
        elif surface_z[1] - tol <= CGbeads[CGbead_index][2] <= surface_z[1] + tol:
            CGbeads_surface_indexes.append(CGbead_index)

    CGbeads_surface_indexes = np.array(CGbeads_surface_indexes)

    # Construct an array of first subsurface CG beads indexes
    CGbeads_first_layer_indexes = []
    for CGbead_index in range(len(CGbeads)):
        if first_layer_z[0] - tol <= CGbeads[CGbead_index][2] <= first_layer_z[0] + tol:
            CGbeads_first_layer_indexes.append(CGbead_index)
        elif first_layer_z[1] - tol <= CGbeads[CGbead_index][2] <= first_layer_z[1] + tol:
            CGbeads_first_layer_indexes.append(CGbead_index)
            
    CGbeads_first_layer_indexes = np.array(CGbeads_first_layer_indexes)
            
    # Construct an array of bulk CG beads (below first subsurface layer)
    CGbeads_bulk_indexes = []
    for CGbead_index in range(len(CGbeads)):
        if not surface_z[0] - tol <= CGbeads[CGbead_index][2] <= surface_z[0] + tol:
            CGbeads_bulk_indexes.append(CGbead_index)
        elif not surface_z[1] - tol <= CGbeads[CGbead_index][2] <= surface_z[1] + tol:
            CGbeads_bulk_indexes.append(CGbead_index)
        elif not first_layer_z[0] - tol <= CGbeads[CGbead_index][2] <= first_layer_z[0] + tol:
            CGbeads_bulk_indexes.append(CGbead_index)
        elif not first_layer_z[1] - tol <= CGbeads[CGbead_index][2] <= first_layer_z[1] + tol:
            CGbeads_bulk_indexes.append(CGbead_index)

    CGbeads_bulk_indexes = np.array(CGbeads_bulk_indexes)
    
    # Construct arrays of surface, first subsurface and bulk CG beads from the main array
    CGbeads_Surface = np.take(AllCGbead_AtomIndexes, CGbeads_surface_indexes)
    CGbeads_FirstLayer = np.take(AllCGbead_AtomIndexes, CGbeads_first_layer_indexes)
    CGbeads_Bulk = np.take(AllCGbead_AtomIndexes, CGbeads_bulk_indexes)
    
    
    return(CGbeads_Surface, CGbeads_FirstLayer, CGbeads_Bulk)

In [7]:
def Initialize(filename, resname, x_points, y_points, z_points):
    
    # Call ReadGRO function for atomic coordinates
    print("Initializing the input...\n")
    
    atoms = np.array(ReadGRO(filename, resname))
    
    print("Reading", filename, "...")
    print(resname, "residue is selected for the bead mapping...")
    print("Total number of atoms in the slab:", len(atoms), "\n")
    
    # Call GenerateCGbeadCenters for CG bead centers and grid spacings
    CGbeads_data = GenerateCGbeadCenters(filename, resname, x_points, y_points, z_points)
    CGbeads = CGbeads_data[0]
    spacings = CGbeads_data[1]
    
    print("Using", x_points,"x", y_points,"x",z_points, "grid for CG centers...")
    print("Using", np.around(spacings, decimals=4), "as a grid spacing vector...")

    
    print(str(len(CGbeads)), "CG centers will be generated...")
    print("Average number of atoms per CG bead is:", round(float(len(atoms))/float(len(CGbeads)), 4))
    print("")
    
    return(atoms, CGbeads, spacings)

In [153]:
def CGBeadMappingMain(filename, resname, x_points, y_points, z_points):
    
    atoms, CGbeads, spacings = Initialize(filename, resname, x_points, y_points, z_points)
    
    """ 
    The idea: 
    0. Create AllCGBead_AtomIndexes
    1. Pick an atom (atoms[atom_number])
    2. Calculate distances between atom and all CG bead centers
    3. Do np.argsort on the distances, pick the first number from np.argsort(distances) - that will
    be the index of the closest CG bead (CGbeads[CGbead_index])!
    5. Start building CGBead_AtomIndexes: AllCGbead_AtomIndexes[CGbead_index].append(atom_number)
    ???
    """
    
    AllCGBead_AtomIndexes = []
    
    # Calculate distances between every atom and all CG beads with the for loop (archived,
    # it is actually not much slower than the NumPy version):
    """AllDistances = []
    
    for atom_number in range(len(atoms)):
        atom_submatrix = np.ones((len(CGbeads), 3)) * atoms[atom_number]
        distances = np.linalg.norm(atom_submatrix - CGbeads, axis = 1)
        AllDistances.append(distances)
        
    print("Distances between every atom and all CG beads have been calculated.\n")"""
    
    # Build a matrix that contains len(CGbeads) repeats of the atomic coordinates
    atoms_matrix = np.repeat(atoms, len(CGbeads), axis=0)
    
    # Reshape the matrix, so that it has len(atoms) elements, each containing len(CGbeads) repeats
    # of the coordinates of atoms[atom_number]
    atoms_matrix = np.reshape(atoms_matrix, (len(atoms), len(CGbeads), 3))
    
    # Build a matrix that contains len(atoms) repeats of all CGbeads coordinates
    CGbeads_matrix = np.repeat([CGbeads], len(atoms), axis=0)
    
    # Calculate all distances
    AllDistances = np.linalg.norm(atoms_matrix - CGbeads_matrix, axis=2)
    print("Distances between every atom and all CG beads have been calculated.\n")
    
    
    
        
    return(AllDistances)    

In [154]:
%%time
AllDistances = CGBeadMappingMain('anatase-101-POPE.gro', '1H151', 16, 16, 9)

Initializing the input...

Reading anatase-101-POPE.gro ...
1H151 residue is selected for the bead mapping...
Total number of atoms in the slab: 13910 

Using 16 x 16 x 9 grid for CG centers...
Using [0.4408 0.4337 0.3884] as a grid spacing vector...
2304 CG centers will be generated...
Average number of atoms per CG bead is: 6.0373

Distances between every atom and all CG beads have been calculated.

CPU times: user 2.46 s, sys: 837 ms, total: 3.3 s
Wall time: 1.13 s


In [155]:
print(AllDistances)

[[7.45101678 7.45817874 7.48551851 ... 2.93058466 3.24126126 3.56721191]
 [7.74428686 7.75087716 7.77688836 ... 2.80108073 3.12391421 3.46026082]
 [4.00822252 3.72183553 3.45543772 ... 7.12202204 7.09366471 7.08651806]
 ...
 [6.82880282 6.65383211 6.49739899 ... 4.06558092 3.99756814 3.96660207]
 [6.06822219 6.08736032 6.1310985  ... 6.28360223 6.44414161 6.62359746]
 [8.36703891 8.22358181 8.09623577 ... 1.82477618 1.66145565 1.57887296]]
